In [4]:
from io_utils import read_fbin, read_ibin
import faiss
print(faiss.__version__)
import numpy as np
file_path = "/workspace/datasets/yandex/base.10M.fbin"
query_path = "/workspace/datasets/yandex/query.public.100K.fbin"
ground_truth_path = "/workspace/datasets/yandex/groundtruth.public.100K.ibin"

1.11.0


In [41]:
data_vector = read_fbin(file_path)
print(type(data_vector))
print(data_vector.ndim, data_vector.shape, data_vector.dtype, data_vector.size)
# print(data_vector[:1])  # Print first 1 elements to verify content

<class 'numpy.ndarray'>
2 (10000000, 200) float32 2000000000


In [42]:
d = data_vector.shape[1]
print(d)
index = faiss.IndexFlatL2(d)  # L2 距离
index2 = faiss.IndexHNSWFlat(d, 128)  # HNSW 索引，32 是 M 参数(每个节点的邻居数)
index.add(data_vector)

# 2. 在添加数据前设置 efConstruction，控制图构建质量
ef_construction = 512
ef_search = 512
index2.hnsw.efConstruction = ef_construction # 添加节点时，创建的搜索列表的大小
index2.add(data_vector)
index2.hnsw.efSearch = ef_search # 查询时使用的候选列表大小

1.11.0
200


In [43]:
query_vector = read_fbin(query_path)
print(type(query_vector))
print(query_vector.ndim, query_vector.shape, query_vector.dtype, query_vector.size)
# print(query_vector[0])  # Print first 3 elements to verify content

<class 'numpy.ndarray'>
2 (100000, 200) float32 20000000


In [44]:
ground_truth_vector = read_ibin(ground_truth_path)
print(type(ground_truth_vector))
print(ground_truth_vector.ndim, ground_truth_vector.shape, ground_truth_vector.dtype, ground_truth_vector.size)
# print(ground_truth_vector[0])  # Print first 3 elements to verify content

<class 'numpy.ndarray'>
2 (100000, 100) int32 10000000


In [45]:
D, I = index.search(query_vector[:1], k=10) # k 是返回的最近邻数量 top k
print("Distances:\n", D)
print("Indices:\n", I)

In [51]:
# index2 = faiss.IndexHNSWFlat(d, 32)  # HNSW 索引，32 是 M 参数
# index2.add(data_vector)
D, I = index2.search(query_vector[1:2], k=100)
print("Distances (HNSW):\n", D)


Distances (HNSW):
 [[1.0129589 1.026768  1.0452527 1.0494717 1.0518775 1.0563109 1.0633299
  1.0637355 1.0646429 1.0652899 1.0676103 1.0694985 1.0702374 1.0716283
  1.0724182 1.07337   1.0736227 1.0744214 1.0748836 1.075975  1.077435
  1.0778415 1.0779184 1.0788741 1.0796036 1.0798122 1.0802459 1.0806906
  1.0812676 1.0823017 1.0834008 1.0841782 1.0850575 1.0854828 1.085919
  1.0860202 1.0870283 1.0873127 1.0873764 1.0877218 1.0878072 1.0881497
  1.088644  1.0889808 1.0890903 1.0891035 1.0893183 1.0904195 1.0904524
  1.0911038 1.0915599 1.0918021 1.0928102 1.0929449 1.0930219 1.0931597
  1.0934038 1.0951505 1.0953696 1.0955584 1.0956061 1.0959105 1.0964681
  1.0966735 1.0967214 1.0969357 1.0969784 1.0970254 1.0975251 1.098405
  1.0985918 1.0991039 1.0991592 1.0992405 1.0994518 1.0996349 1.1001441
  1.1001766 1.1004536 1.1009188 1.1014892 1.1015127 1.1019044 1.1022172
  1.1022496 1.1025505 1.1028004 1.1029263 1.102977  1.1032219 1.1037087
  1.1039796 1.1040254 1.1040876 1.1043749 1.1046

In [52]:
print("Indices (HNSW):\n", I)

Indices (HNSW):
 [[4902414  416781  576754 3671027 8144317 2376157 5037438 3584680 9339720
  2262276  569353 6665820 9658008 3014732 7412880 2844952  745822 2994457
  7519399 4033325 7215739 4427651 7869506 7373781 6502439 2885082 3305295
  5875851 8457055  448730 9455382 9349605 3324944 1886187 6451692   27781
  1844406 9982580 2022557 4384953 9169032  590436 4670891 5045459  320667
  7444475 4641480 2448739 8074853 7251632 9265029 8356512 1874675 7938966
  3348433 1248866 4572642 2509145 9181675 3079728 8753842 5814059 9622136
  2643658 3525001 4678599 7636633 6965774 9256676 8910996 5043827 2581227
  4558147 4707608 6733043  760556  621825 4660919 7767658 2110438 9556181
  3403783 9163783 6031987 5609959 6388113 1821363  627853 3103087 2677399
  7358078 6956164 2864331 5332100 2914088 1236415 5604244 3913104 8096818
  3052304]]


In [53]:
print(ground_truth_vector[1])

[866554089 605877290 838442362 488119882 568179926 297365039 814290707
 568169752 909320134 409471328 966051616 749365818 221720763 984916949
 118964230 167300341 186284176 321529363 422140272 860550809 964598800
 143404192 275782369 595979127 664982983 100427135 729903787 889070901
 426310559 482283752 719074406 563295790 330264223  46874099 386745933
 207331089 348459306  42478173 417683731 990953515 991944347 876176108
 708155914 187710710 874801184  51814217  49374298 711102509 534677561
 751294759 122143620 512667311 396727902 774479427 998504378 625423584
 489098262 802679950 478687408 625384861 780745940 448266271 940784052
 914310858 833860393 415453390 300503259 165650033 388357967 932801677
 278753644 421127702 660723752 738181790  23555098 296853201 764929267
 623793665 406489495 533904862 880031788 858505578 409832119 603630539
 951196187 414078497 151068225 626847097 616539618 455141538 771545745
 895866677 756385106 351436231 251974154 163270134 435345221 378562203
 98472

## Notes
- 现象：结果不同；通过HNSW检索的top 100 不在 groundtruth 中 ()
- 排查：
    - 1，扩大图的边 M=128, ef_construction=512, ef_search=512； 不work；
    - 2，1M数据集太小导致的，换成10M的数据集; 依旧不work； 1B的模型745GB
        - 返回的 ground_truth_vector[i] 是匹配top-k结果的index；10M数据index范围： (0~9999999), 而ground_truth_vector[i]的绝大部分结果index > 9999999  => 超出了子集的数据范围 (1B数据size = 745GB，没有继续尝试)
    - 3，#base.10M.fbin是图像嵌入数据；而query.100k.fbin是文本嵌入数据；原本的search faiss中是使用query 从 图像数据集中选择最匹配的图像#

## Test - 将文本 和 图片同时放到HNSW中
- 也可以看到文本聚集的现象

In [6]:
file_path = "/workspace/datasets/yandex/base.1M.fbin"
query_path = "/workspace/datasets/yandex/query.public.100K.fbin"
data_vector = read_fbin(file_path)
query_vector = read_fbin(query_path)

d = data_vector.shape[1]
index_hnsw = faiss.IndexHNSWFlat(d, 128)  # HNSW 索引，32 是 M 参数(每个节点的邻居数)

# 2. 在添加数据前设置 efConstruction，控制图构建质量
ef_construction = 512
ef_search = 512
index_hnsw.hnsw.efConstruction = ef_construction # 添加节点时，创建的搜索列表的大小
index_hnsw.add(data_vector) # 0-999999
index_hnsw.add(query_vector) # 1000000-1099999
index_hnsw.hnsw.efSearch = ef_search # 查询时使用的候选列表大小

In [7]:
D, I = index_hnsw.search(query_vector[:1], k=100) # k 是返回的最近邻数量 top k
# print("Distances:\n", D)
print("Indices:\n", I)

Indices:
 [[1000000 1086686 1080095 1000845 1063564 1018548 1045208 1004949 1011888
  1026467 1005712 1099107 1086143 1009089 1018661 1005242 1064673 1096411
  1010742 1080212 1078678 1029154 1065670 1077651 1014024 1016135 1093273
  1059438 1098927 1020296 1008932 1031573 1071877 1056880 1052763 1099577
  1036766 1063474 1064063 1017343 1044782 1037544 1049629 1026504  433766
  1050411 1033052 1006617 1056946 1066011  728540 1015975 1068442  851734
   310042  861901 1062235  561154 1019008 1020928  940998 1078840 1025049
   648776 1003146 1035617   57939 1052473 1007586  471427 1098935 1071315
  1027875  329007 1065981 1057407 1084800 1047512  170334 1069565  892479
  1087199 1066884 1030751 1034326 1043420  302213 1017117 1089259 1075591
  1027574 1093492 1073862 1096258 1010188 1007911 1006126   82184 1043389
   330645]]


## Test - 增加随机维度
query_vector